In [1]:
import pandas as pd
# from sklearn.datasets.svmlight_format import load_svmlight_file
from constants import HEADER_TOTAL_EXPOSURE
DATA_DIR = 'testA/'
# X, y = load_svmlight_file(DATA_DIR + 'data.libsvm')

In [2]:
total_exposure = pd.read_csv(DATA_DIR + 'totalExposureLog.out', sep='\t', names=HEADER_TOTAL_EXPOSURE,
                             header=None, index_col=False, quotechar='\t')
total_exposure.drop_duplicates(subset=['req_id', 'ad_loc_id', 'ad_id'], inplace=True)

In [3]:
total_exposure.head()

,req_id,req_ts,ad_loc_id,uid,ad_id,expo_size,bid,pctr,quality_ecpm,total_ecpm
0,53991770,1550409746,94,1160618,451525,50,46,47.217,944.34,3122.340
1,25942318,1550370892,79,203814,214797,64,10,49.094,981.88,1471.880
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584
3,5935886,1550365898,198,7270,160082,64,85,6.123,122.46,642.915
4,11624425,1550361159,168,852707,253902,64,60,4.329,86.58,346.320


In [4]:
total_exposure.req_ts.min()  # 2019-02-16 22:59:27

1550329167

In [6]:
pd.to_datetime(total_exposure.req_ts.min(), unit='s')  # utc时间不对

Timestamp('2019-02-16 14:59:27')

In [7]:
from datetime import timedelta
gap = timedelta(hours=8)
pd.to_datetime(total_exposure.req_ts.min(), unit='s') + gap

Timestamp('2019-02-16 22:59:27')

In [ ]:
utc2bj = timedelta(hours=8)

In [4]:
# std有NaN值！
total_exposure_math_fea = total_exposure.groupby(['ad_id']).describe()

In [5]:
# total_exposure.to_csv(DATA_DIR + 'clean_totalExposureLog.tsv', sep='\t', index=False)
total_exposure_math_fea.head()

ad_loc_id                                                            \
          count        mean        std    min    25%    50%    75%    max   
ad_id                                                                       
1          27.0  188.259259  52.691842   18.0  198.0  198.0  198.0  313.0   
2          75.0  146.560000  92.779343   39.0   56.0  123.0  196.0  297.0   
3          12.0  240.833333  47.668998  209.0  209.0  209.0  297.0  327.0   
4         602.0  208.162791  39.879854   30.0  199.0  209.0  209.0  323.0   
5           1.0   94.000000        NaN   94.0   94.0   94.0   94.0   94.0   

         bid                ...      total_ecpm               uid  \
       count        mean    ...             75%        max  count   
ad_id                       ...                                     
1       27.0  306.629630    ...      1027.25450   5311.008   27.0   
2       75.0  461.933333    ...      3365.49000  80089.520   75.0   
3       12.0   23.166667    ...       425.49675   7855.584   12.0   
4      602.0  173.282392    ...      1684.31250  13872.000  602.0   
5        1.0   12.000000    ...      8572.76000   8572.760    1.0   

                                                                    \
                mean            std       min        25%       50%   
ad_id                                                                
1      615381.777778  476662.410536    4784.0  119111.50  637025.0   
2      700073.173333  422250.156044     253.0  391392.00  695204.0   
3      644729.416667  426314.485286    6518.0  394688.75  666705.0   
4      686511.850498  398081.904489    3110.0  352644.75  685953.5   
5      260267.000000            NaN  260267.0  260267.00  260267.0   

                              
              75%        max  
ad_id                         
1       965421.50  1390924.0  
2       979732.00  1371564.0  
3       833585.75  1363790.0  
4      1022921.25  1395382.0  
5       260267.00   260267.0  

[5 rows x 72 columns]

In [6]:
total_exposure_math_fea.to_hdf(DATA_DIR + 'total_exposure_math_fea.h5', 'total_exposure_math_fea')  # describe太久

In [8]:
bid_fea = total_exposure_math_fea['bid'].reset_index()
bid_fea.columns

Index([u'ad_id', u'count', u'mean', u'std', u'min', u'25%', u'50%', u'75%',
       u'max'],
      dtype='object')

In [9]:
def col_with_prefix(col, prefix, ignore_cols=['ad_id']):
    if col in ignore_cols:
        return col
    return prefix + col
bid_fea.columns = [col_with_prefix(col, 'bid_') for col in bid_fea.columns]
bid_fea.head()

,ad_id,bid_count,bid_mean,bid_std,bid_min,bid_25%,bid_50%,bid_75%,bid_max
0,1,27.0,306.629630,420.721098,24.0,52.50,76.0,414.50,1594.0
1,2,75.0,461.933333,556.746323,68.0,158.00,261.0,522.00,2969.0
2,3,12.0,23.166667,18.393840,10.0,12.75,14.5,23.25,68.0
3,4,602.0,173.282392,132.116900,10.0,86.00,141.5,229.00,1403.0
4,5,1.0,12.000000,NaN,12.0,12.00,12.0,12.00,12.0


In [10]:
ad_size_fea = total_exposure_math_fea['expo_size'].reset_index()
ad_size_fea.columns = [col_with_prefix(col, 'expo_size_') for col in bid_fea.columns]

In [11]:
pctr_fea = total_exposure_math_fea['pctr'].reset_index()
pctr_fea.columns = [col_with_prefix(col, 'pctr_') for col in bid_fea.columns]

In [12]:
ad_size_fea.head()

,ad_id,expo_size_bid_count,expo_size_bid_mean,expo_size_bid_std,expo_size_bid_min,expo_size_bid_25%,expo_size_bid_50%,expo_size_bid_75%,expo_size_bid_max
0,1,27.0,64.0,0.0,64.0,64.0,64.0,64.0,64.0
1,2,75.0,36.0,0.0,36.0,36.0,36.0,36.0,36.0
2,3,12.0,64.0,0.0,64.0,64.0,64.0,64.0,64.0
3,4,602.0,64.0,0.0,64.0,64.0,64.0,64.0,64.0
4,5,1.0,50.0,NaN,50.0,50.0,50.0,50.0,50.0


In [13]:
pctr_fea.head()

,ad_id,pctr_bid_count,pctr_bid_mean,pctr_bid_std,pctr_bid_min,pctr_bid_25%,pctr_bid_50%,pctr_bid_75%,pctr_bid_max
0,1,27.0,3.111519,1.255610,1.225,2.19800,2.9910,3.84550,6.960
1,2,75.0,7.265600,8.182794,1.103,3.08750,6.0540,9.14850,69.826
2,3,12.0,16.047167,25.080070,2.725,5.82125,6.6635,7.78125,89.268
3,4,602.0,7.591223,5.730947,0.010,4.29775,5.9100,9.13175,62.071
4,5,1.0,268.088000,NaN,268.088,268.08800,268.0880,268.08800,268.088


In [14]:
fea_log_stat = bid_fea.merge(ad_size_fea, on='ad_id').merge(pctr_fea, on='ad_id')

In [18]:
from constants import HEADER_AD_STATIC
ad_static_fea = pd.read_csv(DATA_DIR + 'ad_static_feature.out',
                             sep='\t',
                             header=None,
                             names=HEADER_AD_STATIC)

/home/yangxuanhua/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
ad_fea = ad_static_fea.merge(fea_log_stat, on='ad_id',how='outer')

In [20]:
ad_fea.head()

,ad_id,create_ts,account_id,product_id,product_class,industry_id,sizes,bid_count,bid_mean,bid_std,...,expo_size_bid_75%,expo_size_bid_max,pctr_bid_count,pctr_bid_mean,pctr_bid_std,pctr_bid_min,pctr_bid_25%,pctr_bid_50%,pctr_bid_75%,pctr_bid_max
0,106452,1.529959e+09,22226.0,16088,13.0,225,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,233649,1.538222e+09,25681.0,7356,13.0,136,1,1.0,100.000000,NaN,...,1.0,1.0,1.0,0.841000,NaN,0.841,0.84100,0.841,0.84100,0.841
2,547531,1.550731e+09,20696.0,-1,1.0,186,40,41.0,484.365854,92.196192,...,40.0,40.0,41.0,7.248343,5.434941,1.483,2.77700,6.741,8.93700,28.486
3,707841,1.551858e+09,3968.0,-1,3.0,186,40,8.0,205.875000,142.742963,...,40.0,40.0,8.0,5.179125,3.513139,0.905,3.63075,5.098,5.78975,12.008
4,457009,1.550439e+09,23614.0,7447,13.0,172,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
ad_fea.to_csv(DATA_DIR + 'ad_fea.csv', index=None)

In [ ]:
def one_hot_multi_bit(data):
    data = data.apply(lambda x: [int(i) for i in x.split(',') if i != ''])
    length = data.apply(lambda x: len(x))
    max_values = data.apply(max).max() + 1
    data = data.apply(lambda x: sum([2 ** i for i in x]))
    one_hot = data.apply(lambda x: [int(i) for i in bin(x)[2:].rjust(max_values, '0')])
    return one_hot, length

In [ ]:
# todo: 分时段统计曝光、人流量
# 通常广告的曝光性具有上坡性，所以将广告时间分位三个阶段：Early stage， Mid_term stage，late stage(依据在线时间)
# (pd.to_datetime(temp_totalExposureLog_temp['广告请求时间']) - pd.to_datetime(temp_totalExposureLog_temp['创建时间'] )).days